In [1]:
import hashlib
import json
import os
from heuristique.genetique import *
from objects.solution import Solution
from objects.thread import bourlingueur
from script.affichage import plot_gantt, net_voisinage
from script.data import loader 
import numpy as np
import networkx as nx 
import copy
import matplotlib.pyplot as plt
import random
from heuristique.heuristique_gloutonne import heuristique_gloutone
from heuristique.ordo_avec_liste import alloc_avec_liste, allocation_dispomachine_ASAP,decodage_bier
from Results.summary import main
from script.utils import vecteur_bier
from script.recherche_voisin import *

data, optimum= loader('ft10')

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [ ]:
main_genetique('ft20',selec = 20, crois=200, gene= 1000)

In [4]:
import multiprocess as mp
output = mp.Queue()

for i in range(3):
    processes = [mp.Process(target= main_genetique, args= ('la01', 20, 200, 1000))for x in range(4)]
    
    
    for p in processes:
        p.start()
        
    for p in processes:
        p.join()

In [ ]:
    
    
att = {'nom' : 'ft20', 'selec': 20, 'crois':200, 'gene': 1000}
# Création des threads
thread_1 = bourlingueur(**att)
thread_2 = bourlingueur(**att)
thread_3 = bourlingueur(**att)

# Lancement des threads
thread_1.start()
thread_2.start()
thread_3.start()
# Attend que les threads se terminent
thread_1.join()
thread_2.join()
thread_3.join()



In [5]:
chemin='/home/pa/PycharmProjects/AP_JOB_SHOP/Results/génétique ft/ft10'
t = main('/home/pa/PycharmProjects/AP_JOB_SHOP/Results/génétique ft/la01')

In [ ]:
for file in os.listdir(chemin):
    print(file)

In [ ]:
chemin = os.getcwd()
chemin

In [ ]:
import operator
data, optimum= loader('ft06')
instltp = Instance(**data)
heuristique_gloutone(instltp, prio = 'SPT')
liste= vecteur_bier(instltp)


sol = Solution(data, liste, root= "None")
sol.voisinage()

temp = []
index = []
for i, v in enumerate(sol.voisin):
    if v in temp:
        temp.append(v)
        index.append(i)
    else:
        sol.voisin.remove(v)

best = sorted(sol.voisin, key=operator.attrgetter('makeSpan'))

In [ ]:
import hashlib
import json
from heuristique.genetique import *
from objects.population import Population
from objects.solution import Solution
from script.utils import sort_task, decodage_gene
from objects.instance import Instance
from script.affichage import plot_gantt, net_voisinage
from script.data import loader 
import numpy as np
import networkx as nx 
import copy
import matplotlib.pyplot as plt
import random
import objects.logger
from heurita0stique.heuristique_gloutonne import heuristique_gloutone
from heuristique.ordo_avec_liste import alloc_avec_liste, allocation_dispomachine_ASAP,decodage_bier
from script.utils import vecteur_bier
from script.recherche_voisin import *

data, optimum= loader('ft10')
meth = [inversion_all, decalage_all, permutation_by_2_all]


In [ ]:
data, optimum= loader('ft06')
instltp = Instance('ft20')
heuristique_gloutone(instltp, prio = 'SPT')
liste= vecteur_bier(instltp)


sol = Solution(instltp, liste, root= "None")

g = exploration_voisinage(sol, n = 2 , max_depth = 30, crit_stagnation=40)



In [ ]:
data, optimum= loader('ft06')
instltp = Instance('ft20')
heuristique_gloutone(instltp, prio = 'LPT')
liste= vecteur_bier(instltp)
sol = Solution(instltp, liste, root= "None")

In [ ]:
sol.voisinage()

In [ ]:
for s in sol.voisin:
    print(s.nom, s.makeSpan)